In [81]:
import simpy
import random
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

np.random.seed(42)

In [82]:
class FiFoServers:
    def __init__(self, env, mu, num_servers):
        self.mu = mu
        self.server = simpy.Resource(env, capacity=num_servers)
        self.processing_times = []
        self.final_arrival = 1
        self.N = 0
        self.queue_length = 0

class SJFServers:
    def __init__(self, env, mu, num_servers):
        self.mu = mu
        self.server = simpy.PriorityResource(env, capacity=num_servers)
        self.processing_times = []
        self.final_arrival = 1
        self.N = 0
        self.queue_length = 0

def task(name, env, servers, processing_time, waiting_times, print_tasks):

    # A task arrives at the server
    start_time = env.now
    if print_tasks:
        print(f'Task {name} arriving at {start_time}')
    
    servers.final_arrival = start_time
    servers.N += 1
    servers.queue_length += 1

    # Request task to server
    with servers.server.request() as req:

        # Once there is availability at the server, initiate task
        yield req
        end_time = env.now
        servers.queue_length -= 1
  
        if print_tasks:
            print(f'Server starts processing {name} at {end_time}')

        yield env.timeout(processing_time)

        if print_tasks:
            print(f'Server done processing {name} at {env.now}')

        waiting_times.append(end_time - start_time)
        servers.processing_times.append(processing_time)


def task_generator(env, servers, N, lambda_, waiting_times, print_tasks=False):

    # Generate N tasks
    for i in range(N):
        
        # Calculate time at server by Poisson process, not sure if necessary
        processing_time = max(0, int(random.expovariate(servers.mu)))

        # Create task
        env.process(task(i, env, servers, processing_time, waiting_times, print_tasks))

        # Calculate time until next task by Poisson process, not sure if necessary
        next_arrival = max(0, int(random.expovariate(lambda_)))
        yield env.timeout(next_arrival)


In [83]:
def get_confidence(data, p=0.95):
    """
    Calculate the mean and confidence interval of a dataset for a given confidence level.

    Parameters:
    - data (array-like): Input data.
    - p (float, optional): Confidence level (default is 0.95).

    Returns:
    tuple: A tuple containing the mean and the confidence interval.
    """

    mean = np.mean(data)
    n = len(data)
    lamb = stats.t.ppf((1 + p) / 2, n - 1)
    sigma = np.std(data)
    confidence = (lamb * sigma / np.sqrt(n))
    return mean, confidence

In [84]:
def run_simulation(num_servers, mu, lambda_, N, tmax, scheduler):
    env = simpy.Environment()

    # Use either First-in First-out or Shortest Job First scheduling
    if scheduler.lower() == 'fifo':
        servers = FiFoServers(env, mu, num_servers)
    elif scheduler.lower() == 'sjf':
        servers = SJFServers(env, mu, num_servers)

    waiting_times = []

    # Generate tasks
    env.process(task_generator(env, servers, N, lambda_ * num_servers, waiting_times))
    env.run(tmax)

    # Calculate statistics on waiting times
    waiting_times_array = np.array(waiting_times)
    std_waiting_time = np.std(waiting_times_array)
    
    confidence_level = 0.95

    # Calculate confidence interval of waiting time
    mean, confidence = get_confidence(waiting_times_array, confidence_level)
    confidence_interval = (mean - confidence, mean + confidence)

    print(f"{num_servers} servers:")
    print(f"Average Waiting Time: {mean}")
    print(f"The analytical average waiting time is: {1 / (servers.mu * (1 - lambda_/servers.mu) * (lambda_ / servers.mu))}")
    print(f"Standard Deviation of Waiting Time: {std_waiting_time}")
    print(f"Confidence Interval (95%): {confidence_interval}\n")

    # Calculate statistics on processing times
    processing_times_array = np.array(servers.processing_times)
    processing_rates_array = 1 / np.where(processing_times_array == 0, 0.1, processing_times_array)
    std_processing_rate= np.std(processing_rates_array)

    # Calculate confidence interval of processing rate
    mean, confidence = get_confidence(processing_rates_array, confidence_level)
    confidence_interval = (mean - confidence, mean + confidence)

    print(f"Average Processing Rate of server: {mean}")
    print(f"Standard deviation of Processing Rate {std_processing_rate}")
    print(f"Confidence Interval (95%): {confidence_interval}\n")

    # Print statsitics on arrivals
    print(f"Time of final arrival: {servers.final_arrival}")
    print(f"Number of arrivals: {servers.N}")
    print(f"Average arrivals per timestep: {servers.N / servers.final_arrival}\n\n")

# Parameters
mu = 0.02
lambda_ = 0.015
N = 100000
tmax = 5000000
num_servers_list = [1, 2, 4]
scheduler = 'fifo'

# Loop over number of servers
for num_servers in num_servers_list:
    run_simulation(num_servers, mu, lambda_, N, tmax, scheduler)

1 servers:
Average Waiting Time: 148.04470183334436
The analytical average waiting time is: 266.6666666666667
Standard Deviation of Waiting Time: 191.98872258925752
Confidence Interval (95%): (146.67562473547116, 149.41377893121756)

Average Processing Rate of server: 0.2621285479529168
Standard deviation of Processing Rate 1.3455305254627656
Confidence Interval (95%): (0.25253353110996685, 0.27172356479586673)

Time of final arrival: 4999975
Number of arrivals: 75547
Average arrivals per timestep: 0.015109475547377738


2 servers:
Average Waiting Time: 67.67068
The analytical average waiting time is: 266.6666666666667
Standard Deviation of Waiting Time: 96.68617578711861
Confidence Interval (95%): (67.07141663220285, 68.26994336779715)

Average Processing Rate of server: 0.2729242243198067
Standard deviation of Processing Rate 1.3810167495359147
Confidence Interval (95%): (0.2643646475077778, 0.2814838011318356)

Time of final arrival: 3278182
Number of arrivals: 100000
Average arriva

In [85]:
scheduler = 'sjf'

for num_servers in num_servers_list:
    run_simulation(num_servers, mu, lambda_, N, tmax, scheduler)

1 servers:
Average Waiting Time: 158.17225992131597
The analytical average waiting time is: 266.6666666666667
Standard Deviation of Waiting Time: 203.59808408365132
Confidence Interval (95%): (156.72232381113952, 159.62219603149242)

Average Processing Rate of server: 0.2738281631749595
Standard deviation of Processing Rate 1.386487634201539
Confidence Interval (95%): (0.2639542073556308, 0.2837021189942882)

Time of final arrival: 4999978
Number of arrivals: 75747
Average arrivals per timestep: 0.015149466657653293


2 servers:
Average Waiting Time: 66.3338
The analytical average waiting time is: 266.6666666666667
Standard Deviation of Waiting Time: 94.0602928847237
Confidence Interval (95%): (65.75081192097831, 66.91678807902169)

Average Processing Rate of server: 0.28063624006088717
Standard deviation of Processing Rate 1.4077005201974806
Confidence Interval (95%): (0.27191127656206465, 0.2893612035597097)

Time of final arrival: 3282749
Number of arrivals: 100000
Average arrivals 